# Algotic Economic Data Crawler - Colab Version

این نوت‌بوک همان عملکرد اسکریپت اصلی را دارد اما برای محیط Google Colab تنظیم شده است.

In [ ]:
# نصب کتابخانه‌های مورد نیاز
!pip install selenium beautifulsoup4 pandas webdriver_manager

In [ ]:
# اتصال گوگل درایو
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd
from bs4 import BeautifulSoup

# تنظیمات درایور برای Colab
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')

# راه‌اندازی درایور با webdriver_manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# باز کردن صفحه
driver.get("https://algoticlab.com/calendar.html")

time.sleep(7)  # صبر برای لود کامل صفحه

try:
    # پیدا کردن iframe
    iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "iframe"))
    )
    driver.switch_to.frame(iframe)
    time.sleep(3)  # صبر برای لود محتوای iframe
    try:
        table = driver.find_element(By.ID, "ecEventsTable")
        print("Table found!\n")
        # استخراج داده با BeautifulSoup
        soup = BeautifulSoup(table.get_attribute('outerHTML'), 'html.parser')
        rows = soup.find_all('tr')
        data = []
        for row in rows:
            cols = row.find_all('td')
            if not cols:
                continue
            # استخراج اهمیت خبر
            imp_value = ''
            for idx, col in enumerate(cols):
                if 'sentiment' in col.get('class', []):
                    # اگر تعطیلی بود
                    span = col.find('span', class_='bold')
                    if span and span.text.strip() == 'Holiday':
                        imp_value = 'Holiday'
                    else:
                        icons = col.find_all('i', class_='newSiteIconsSprite')
                        full = sum('grayFullBullishIcon' in i.get('class', []) for i in icons)
                        empty = sum('grayEmptyBullishIcon' in i.get('class', []) for i in icons)
                        if full == 3:
                            imp_value = 'RED'
                        elif full == 2 and empty == 1:
                            imp_value = 'Orange'
                        elif full == 1 and empty == 2:
                            imp_value = 'Yellow'
                    # جایگزینی مقدار اهمیت در ستون مربوطه
                    cols[idx] = imp_value
            cols = [col.text.strip() if hasattr(col, 'text') else col for col in cols]
            data.append(cols)
        # استخراج هدرها
        headers = []
        header_row = soup.find('thead').find('tr')
        for th in header_row.find_all('th'):
            headers.append(th.text.strip())
        # ذخیره به صورت DataFrame و خروجی گرفتن
        df = pd.DataFrame(data, columns=headers)
        
        # ذخیره فایل‌ها در گوگل درایو
        output_dir = '/content/drive/MyDrive/AlgoticData/'
        import os
        os.makedirs(output_dir, exist_ok=True)
        
        df.to_csv(f'{output_dir}economic_calendar.csv', index=False, encoding='utf-8-sig')
        print("Data saved to economic_calendar.csv in Google Drive.")
        
        df.to_json(f'{output_dir}economic_calendar.json', orient='records', force_ascii=False, indent=2)
        print("Data saved to economic_calendar.json in Google Drive.")
        
        with open(f'{output_dir}economic_calendar.md', 'w', encoding='utf-8') as f:
            f.write(df.to_markdown(index=False))
        print("Data saved to economic_calendar.md in Google Drive.")
        
        # نمایش داده‌ها در نوت‌بوک
        from IPython.display import display
        display(df)
        
    except Exception as e:
        print("Table with ID ecEventsTable not found or error:", e)
    driver.switch_to.default_content()
except Exception as e:
    print("Error finding or switching to iframe:", e)

driver.quit()